# Premises > Premises > Conclusions


In [3]:
import os
import xml.etree.ElementTree as ET
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout

def sanitize_id(node_id):
    """Ensure node IDs are safe for Graphviz by replacing problematic characters"""
    return node_id.replace('%', 'pct_').replace('|', '_or_')

def visualize_argument_graph(xml_path, output_dir):
    # Parse XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Create directed graph
    G = nx.DiGraph()
    id_mapping = {}  # Map original IDs to sanitized versions
    
    # Add nodes with sanitized IDs
    for elem in root.findall('.//prem') + root.findall('.//conc'):
        orig_id = elem.attrib['ID']
        safe_id = sanitize_id(orig_id)
        id_mapping[orig_id] = safe_id
        arg_type = 'premise' if elem.tag == 'prem' else 'conclusion'
        G.add_node(safe_id, type=arg_type, orig_id=orig_id)
    
    # Add relationships using sanitized IDs
    for elem in root.findall('.//prem') + root.findall('.//conc'):
        source_id = id_mapping[elem.attrib['ID']]
        
        # Process support relationships
        if 'SUP' in elem.attrib:
            for target in elem.attrib['SUP'].split('|'):
                if target in id_mapping:  # Ensure target exists
                    G.add_edge(source_id, id_mapping[target], 
                              relationship='support', color='green')
        
        # Process attack relationships
        if 'ATT' in elem.attrib:
            for target in elem.attrib['ATT'].split('|'):
                if target in id_mapping:
                    G.add_edge(source_id, id_mapping[target], 
                              relationship='attack', color='red')
    
    # Create hierarchical layout
    try:
        pos = graphviz_layout(G, prog='dot')
    except TypeError:
        # Fallback to spring layout if Graphviz fails
        pos = nx.spring_layout(G, seed=42)
    
    # Prepare visualization
    node_colors = ['lightblue' if G.nodes[n]['type'] == 'premise' else 'lightgreen' 
                   for n in G.nodes]
    edge_colors = [G.edges[e]['color'] for e in G.edges]
    labels = {n: G.nodes[n]['orig_id'] for n in G.nodes}  # Use original IDs for labels
    
    # Draw graph
    plt.figure(figsize=(15, 10))
    nx.draw(G, pos, labels=labels, with_labels=True, node_size=1500, 
            node_color=node_colors, edge_color=edge_colors, 
            font_size=9, arrowsize=20)
    
    # Create legend
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightblue', 
                   markersize=10, label='Premise'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightgreen', 
                   markersize=10, label='Conclusion'),
        plt.Line2D([0], [0], color='green', lw=2, label='Support Relationship'),
        plt.Line2D([0], [0], color='red', lw=2, label='Attack Relationship')
    ]
    plt.legend(handles=legend_elements, loc='upper right')
    
    # Save output
    filename = os.path.basename(xml_path).replace('.xml', '_graph.png')
    plt.savefig(os.path.join(output_dir, filename), bbox_inches='tight')
    plt.close()

# Process all XML files
input_dir = 'xml_files/all_xml'
output_dir = 'graphs2'

for xml_file in os.listdir(input_dir):
    if xml_file.endswith('.xml'):
        visualize_argument_graph(
            os.path.join(input_dir, xml_file),
            output_dir
        )
